<a href="https://colab.research.google.com/github/oliverZZ/193B_Glucose_Monitor/blob/master/Infection_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import pandas as pd
pd.options.display.max_rows = 20
import matplotlib.pyplot as plt
import numpy as np

bg = pd.read_excel('/content/drive/My Drive/EEC 193/EEC 193B/Burn_Glucose.xlsx', skiprows = 0)

In [44]:
bg

,PER_CODE,Collection Date,Time Vitals,Systolic,Diastolic,MAP,HR,RR,Temp,CVP,GCS,Vent,Time CBC,WBC,Hgb,Hct,Platelet,Time Labs,Na,K+,BUN,Creatnine,Glucose,Tbili,Chloride,V_CO2,PaO2,FIO2,PaCO2,HCO3,PH,Sepsis
0,21,2011-01-02,614.0,113.0,67.0,78.0,102.0,15.0,37.0,7.0,15.0,No,5.0,6.9,8.1,24.5,578.0,5.0,136.0,3.6,7.0,0.53,110.0,0.3,101.0,29.0,NaN,NaN,NaN,NaN,NaN,0.0
1,21,2011-01-03,600.0,137.0,67.0,85.0,105.0,16.0,37.3,7.0,15.0,No,59.0,7.8,7.9,25.1,615.0,59.0,137.0,3.7,8.0,0.55,130.0,0.3,104.0,27.0,NaN,NaN,NaN,NaN,NaN,0.0
2,21,2011-01-04,1000.0,124.0,56.0,73.0,104.0,18.0,37.5,7.0,15.0,No,208.0,8.9,8.7,26.6,607.0,208.0,135.0,3.4,8.0,0.53,172.0,0.3,103.0,24.0,NaN,NaN,NaN,NaN,NaN,0.0
3,21,2011-01-05,930.0,143.0,69.0,85.0,104.0,18.0,37.5,7.0,15.0,No,103.0,9.3,8.4,25.7,414.0,103.0,134.0,4.8,10.0,0.49,150.0,0.3,103.0,25.0,NaN,0.21,NaN,NaN,NaN,0.0
4,21,2011-01-06,1130.0,130.0,59.0,83.0,104.0,23.0,38.0,7.0,15.0,No,130.0,8.4,8.9,27.7,701.0,130.0,136.0,3.5,11.0,0.51,109.0,0.3,103.0,27.0,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6536,PCR-004-00018,2013-05-27,600.0,140.0,70.0,102.0,116.0,26.0,38.5,NaN,NaN,No,211.0,17.6,9.2,26.8,454.0,211.0,138.0,4.5,12.0,0.88,139.0,NaN,103.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN
6537,PCR-004-00018,2013-05-28,600.0,158.0,53.0,82.0,110.0,22.0,37.8,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6538,PCR-004-00018,2013-05-29,600.0,133.0,51.0,72.0,120.0,22.0,38.5,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6539,PCR-004-00018,2013-05-30,600.0,118.0,49.0,73.0,118.0,22.0,38.8,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Preprocessing ##

In [0]:
def Drop_nan_row(df):
  
  drop_idx = []
  for i in range(len(df)):
    nan_count = 0
    for x in df.loc[i]:
      if str(x) == 'nan':
        nan_count += 1
    if nan_count >= 15:
      drop_idx.append(i)
    
  modified_df = df.drop(df.index[drop_idx], errors = 'ignore')
  modified_df = modified_df.reset_index(drop = True)

  return modified_df

In [0]:
def Drop_unrelated_cols(df):
  df = df.drop(columns = ['PER_CODE', 'Collection Date', 'Time Vitals', 'Time Labs'], errors = 'ignore')
  return df

In [0]:
def Drop_nan_col(df):
  num_rows = len(df)
  for x in df:
    num_nan = 0
    for i in df[x]:
      if str(i) == 'nan':
        num_nan += 1
    print(x, num_nan/num_rows);
  
  return df

In [0]:
def data_preprocess(df):

  # drop rows that have too many nan
  df = Drop_nan_row(df)
  # drop times and patient ids
  df = Drop_unrelated_cols(df)
  # drop cols that have too many nan
  

  return df

In [0]:
bg = data_preprocess(bg)

In [57]:
bg = Drop_nan_col(bg)

Systolic 0.0005446623093681918
Diastolic 0.0005446623093681918
MAP 0.0039941902687000725
HR 0.00036310820624546115
RR 0.0018155410312273058
Temp 0.0018155410312273058
CVP 0.38852578068264343
GCS 0.023965141612200435
Vent 0.004720406681190995
Time CBC 0.013071895424836602
WBC 0.013798111837327523
Hgb 0.014161220043572984
Hct 0.01289034132171387
Platelet 0.01579520697167756
Na 0.004175744371822803
K+ 0.004175744371822803
BUN 0.00980392156862745
Creatnine 0.005083514887436456
Glucose 0.0039941902687000725
Tbili 0.6198257080610022
Chloride 0.005809731299927378
V_CO2 0.006172839506172839
PaO2 0.48474945533769065
FIO2 0.3070079883805374
PaCO2 0.4845679012345679
HCO3 0.4843863471314452
PH 0.4843863471314452
Sepsis 0.874001452432825


In [51]:
bg

,Systolic,Diastolic,MAP,HR,RR,Temp,CVP,GCS,Vent,Time CBC,WBC,Hgb,Hct,Platelet,Na,K+,BUN,Creatnine,Glucose,Tbili,Chloride,V_CO2,PaO2,FIO2,PaCO2,HCO3,PH,Sepsis
0,113.0,67.0,78.0,102.0,15.0,37.0,7.0,15.0,No,5.0,6.9,8.1,24.5,578.0,136.0,3.6,7.0,0.53,110.0,0.3,101.0,29.0,NaN,NaN,NaN,NaN,NaN,0.0
1,137.0,67.0,85.0,105.0,16.0,37.3,7.0,15.0,No,59.0,7.8,7.9,25.1,615.0,137.0,3.7,8.0,0.55,130.0,0.3,104.0,27.0,NaN,NaN,NaN,NaN,NaN,0.0
2,124.0,56.0,73.0,104.0,18.0,37.5,7.0,15.0,No,208.0,8.9,8.7,26.6,607.0,135.0,3.4,8.0,0.53,172.0,0.3,103.0,24.0,NaN,NaN,NaN,NaN,NaN,0.0
3,143.0,69.0,85.0,104.0,18.0,37.5,7.0,15.0,No,103.0,9.3,8.4,25.7,414.0,134.0,4.8,10.0,0.49,150.0,0.3,103.0,25.0,NaN,0.21,NaN,NaN,NaN,0.0
4,130.0,59.0,83.0,104.0,23.0,38.0,7.0,15.0,No,130.0,8.4,8.9,27.7,701.0,136.0,3.5,11.0,0.51,109.0,0.3,103.0,27.0,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,130.0,60.0,82.0,120.0,21.0,38.0,NaN,NaN,No,322.0,4.9,11.7,35.0,100.0,135.0,3.8,14.0,0.75,120.0,NaN,105.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN
5504,115.0,52.0,72.0,115.0,19.0,38.2,NaN,NaN,Yes,528.0,11.3,8.3,25.3,135.0,137.0,4.2,16.0,0.89,128.0,NaN,106.0,30.0,131,0.4,39.0,28.0,7.47,NaN
5505,132.0,70.0,88.0,114.0,22.0,37.4,NaN,NaN,No,157.0,19.3,6.9,21.2,174.0,136.0,3.9,11.0,0.78,126.0,0.1,104.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN
5506,129.0,60.0,81.0,98.0,18.0,37.0,NaN,NaN,No,418.0,28.6,6.8,20.1,423.0,140.0,4.5,15.0,0.95,79.0,NaN,104.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.6.2)


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import imblearn

x = bg[['RR', 'WBC', 'Glucose', 'Platelet']].values
y = bg['Sepsis'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

balance = imblearn.over_sampling.SMOTE()
train_x, train_y = balance.fit_resample(x_train, y_train)
train_y = pd.Series(train_y)

model = RandomForestClassifier()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

y_pred = model.predict(x_test)

In [0]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.46      0.43      0.44        14
         1.0       0.58      0.61      0.59        18

    accuracy                           0.53        32
   macro avg       0.52      0.52      0.52        32
weighted avg       0.53      0.53      0.53        32

